# Predicting Cancer Type from Single Cell RNA-Seq Data

## Dataloader

### Elsa Bismuth & Jonathan Mathews

Updated On: 11/09/2022

# Import Statements

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

# Collecting and Processing RNA-Seq Data

  - Converts each file into a dataframe
  - Finds common genes across all datasets

In [ ]:
DATADIR = "/content/drive/My Drive/ML4FG_Project/Data/"
numerizeClass = {'AML': 0, 'CRC': 1, 'GBM': 2, 'LUAD': 3, 'MPAL': 4, 'PDAC': 5}
allGenes = []
fileNames = []
cancerLabels = []
classDist = []

for tsvFile in os.listdir(DATADIR):

  if tsvFile.endswith('.tsv'):
    
    fileNames.append(tsvFile)

    data = pd.read_csv(os.path.join(DATADIR, tsvFile), sep = '\t')
    data = set(data.index)
    allGenes.append(data)

    cancerLabels.append(tsvFile.split('-')[0])
    classDist.append(numerizeClass[ tsvFile.split('-')[0] ])

    print(len(allGenes), len(cancerLabels))

1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,6,8,9,13,17,18,21,23,24,25,26,30,31,33,34,36,37,38,42,43,44,45,46,47,48,49,50,52,53,55,57,59,60,61,65,66,67,71,73,74,76,80,81,83,84,85,86,87,91,92,94,96,98,99,100,101,102,103,104,106,107,109,110,111,112,113,116,117,118,119,120,123,125,126,128,129,130,131,132,133,134,135,136,138,140,142,143,144,146,147,150,153,154,155,157,158,159,160,161,162,164,165,167,168,169,171,173,177,179,181,182,184,186,187,188,189,192,193,194,196,200,201,202,203,204,205,206,207,208,209,210,212,213,214,215,216,217,218,219,222,225,226,228,229,230,231,232,233,234,237,240,241,242,243,244,245,246,247,248,249,250,251,252,255,259,261,264,265,266,267,270,271,272,273,276,279,280,282,283,284,285,286,287,289,290,291,293,295,296,297,299,300,301,302,303,304,305,306,307,309,310,311,312,314,315,316,317,318,321,322,323,324,325,326,328,329,332,334,335,337,339,340,341,344,345,346,347,348,349,350,353,354,355,357,358,359,36

16 16


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,4,5,7,8,9,10,11,12,13,14,15,16,17,18,20,21,22,23,25,29,30,31,34,35,36,37,38,39,40,41,43,45,46,47,49,50,51,52,54,56,57,58,59,63,64,67,68,71,72,73,74,75,76,77,81,82,89,90,91,92,93,94,95,96,97,98,99,100,102,104,105,108,109,112,114,115,117,118,119,120,121,122,124,125,127,128,129,132,134,135,136,137,138,139,140,141,142,143,145,146,147,148,151,152,153,156,157,158,159,160,161,163,165,166,168,169,170,172,175,177,178,180,183,186,188,189,191,192,194,195,196,198,199,201,202,203,204,207,209,210,211,212,213,214,215,217,219,221,222,223,224,226,227,228,229,230,231,232,236,237,238,239,240,241,242,243,244,249,250,251,252,253,255,256,257,258,259,262,263,264,265,270,271,272,273,275,277,278,280,281,283,284,285,287,288,289,294,295,297,298,299,300,301,302,303,304,305,306,308,310,311,312,314,315,317,318,319,320,321,322,324,327,328,329,330,331,332,333,334,336,337,338,339,340,341,342,343,344,345,347,349,350

17 17


...

28 28


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,3,5,6,7,8,9,10,13,14,15,18,19,21,23,24,25,26,27,28,29,32,33,34,36,37,38,39,40,41,44,45,47,48,49,50,51,52,54,55,57,58,59,60,61,62,63,64,65,66,68,70,71,72,73,75,78,79,80,81,82,83,84,85,86,87,88,90,91,93,95,96,97,98,99,101,102,107,109,110,112,113,116,117,119,120,122,123,124,125,126,127,128,129,131,133,134,135,138,139,144,146,148,149,150,151,153,155,158,159,160,162,165,166,167,169,176,177,179,180,181,182,184,185,186,187,189,190,191,192,193,194,195,197,198,200,201,202,204,205,206,207,209,211,212,214,218,219,220,222,223,224,225,226,227,228,229,230,232,234,235,236,237,238,241,245,246,247,248,250,251,252,253,254,255,256,257,258,261,262,265,266,267,268,270,271,272,273,274,275,276,278,279,280,282,283,284,285,286,287,288,289,290,291,292,293,295,296,297,298,299,300,301,302,304,306,307,308,310,313,314,315,318,319,323,324,325,327,328,330,335,336,338,339,340,341,344,345,346,347,348,349,350,351,353,3

29 29


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,4,5,6,7,10,11,13,15,16,17,18,20,21,22,23,24,26,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,57,58,59,60,61,63,64,66,67,68,69,72,74,75,76,77,78,80,81,82,83,85,87,91,93,95,97,99,102,105,108,109,110,111,112,113,114,115,117,119,120,121,123,126,127,128,130,131,132,133,134,135,136,137,139,140,141,142,143,144,145,146,147,148,152,155,157,158,159,161,163,164,167,168,169,170,171,174,175,176,177,178,179,181,182,183,185,186,188,190,191,192,193,194,195,196,197,198,199,200,202,203,205,208,210,213,214,215,216,217,218,219,220,221,225,227,228,229,232,240,242,243,244,245,246,247,248,249,250,251,253,254,255,259,260,261,263,265,267,268,269,270,271,272,273,274,275,277,278,282,283,284,285,287,288,289,290,291,292,294,295,296,298,299,300,301,302,303,304,306,307,308,309,311,312,313,315,316,317,319,321,322,323,324,327,328,329,330,332,333,334,335,336,337,338,340,342,344,345,3

30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84


# Split Data into Train/Validation/Test Sets

- 50% Training
- 25% Validation
- 25% Testing
- Stratifies across cancer label to give each dataset the same class distribution

In [ ]:
filesTrainVal, filesTest, yTrainVal, yTest = train_test_split(fileNames,
                                                              classDist,
                                                              test_size = 0.25,
                                                              random_state = 42,
                                                              shuffle = True,
                                                              stratify = classDist)

filesTrain, filesVal, yTrain, yVal = train_test_split(filesTrainVal,
                                                      yTrainVal,
                                                      test_size = 0.33,
                                                      random_state = 42,
                                                      shuffle = True,
                                                      stratify = yTrainVal)

# Prepare Files for Dataloader

- Stores the filenames of each dataset
- Allows for easy input to a dataloader in PyTorch

In [ ]:
with open('/content/drive/My Drive/ML4FG_Project/Train_Files.txt', 'w') as f:

  for trainFile in filesTrain:

    f.write('%s\n' % trainFile)

with open('/content/drive/My Drive/ML4FG_Project/Validation_Files.txt', 'w') as f:

  for valFile in filesVal:

    f.write('%s\n' % valFile)

with open('/content/drive/My Drive/ML4FG_Project/Test_Files.txt', 'w') as f:

  for testFile in filesTest:

    f.write('%s\n' % testFile)

# Find and Store Common Genes

- Common genes between all datasets found to allow machine learning models to properly run

In [ ]:
commonGenes = list(set.intersection(*allGenes))

In [ ]:
with open('/content/drive/My Drive/ML4FG_Project/Common_Genes.txt', 'w') as f:

  f.write('%s\n' % len(commonGenes))
  for gene in commonGenes:

    f.write('%s\n' % gene)

In [ ]:
cancerLabelDF = pd.Series(cancerLabels)
cancerClassCounts = cancerLabelDF.value_counts()
cancerClassCounts.to_csv('/content/drive/My Drive/ML4FG_Project/Class_Labels_Distribution.csv', index = True)